### link to part [PART 1 model training](https://www.kaggle.com/anantgupt/cassava-leaf-doctor-model-training-keras)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import glob

In [ ]:
json_path = '../input/cassava-competition-trained-models/leaf-doctor-model-resnet50.json'
model_weights = '../input/cassava-competition-trained-models/leaf-doctor-wieghts-resnet50.hdf5'

with open(json_path, 'r') as json_file:
    json_savedModel = json_file.read()
    
model = tf.keras.models.model_from_json(json_savedModel)
model.load_weights(model_weights)

In [ ]:
model.summary()

In [ ]:
test_images = glob.glob('../input/cassava-leaf-disease-classification/test_images/*.jpg')
print(test_images)

In [ ]:
df = pd.DataFrame(np.array(test_images), columns=['Path'])
df

In [ ]:
df = tf.data.Dataset.from_tensor_slices((df.Path.values))

row, col = 512,  512

def process(image_path):
    # load the raw data from the file as a string
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.random_brightness(img, 0.3)
    img = tf.image.random_flip_left_right(img, seed=None)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_crop(img, size=[row,col, 3])
    return img
    
df = df.map(process, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(8)

In [ ]:
preds = []
for i in range(5):
    
    pred_test = model.predict(df, workers=16, verbose=1)
    preds.append(pred_test)

In [ ]:
pred = np.mean(preds, axis=0)
pred

In [ ]:
pred = np.argmax(pred, axis=-1)

In [ ]:
sub = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
sub['label'] = pred
sub

In [ ]:
sub.to_csv('submission.csv', index=False)